<a href="https://colab.research.google.com/github/fatemafaria142/Analyzing-Large-Language-Models-with-LangChain/blob/main/ConversationBufferMemory_for_Mental_Health_Advice_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Install Required Packages**

In [1]:
!pip install accelerate peft bitsandbytes transformers trl datasets torch langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.3/230.3 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

### **Initializing the Model**
* Load the model using a 4-bit configuration, employing double quantization, and set bfloat16 as the compute data type.

* Notably, we opt for the instruct-tuned model in this instance rather than the base model. It's worth mentioning that fine-tuning a base model necessitates a more substantial amount of data!

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
    )


* **Model and Tokenizer:** https://huggingface.co/openchat/openchat-3.5-1210

In [4]:
mode_id = "openchat/openchat-3.5-1210"

In [5]:
model = AutoModelForCausalLM.from_pretrained(
        mode_id, quantization_config=bnb_config, device_map="auto", use_cache=False
    )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/623 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [6]:
tokenizer = AutoTokenizer.from_pretrained("openchat/openchat-3.5-1210")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


**Documentation Link:**
* https://python.langchain.com/docs/integrations/llms/huggingface_pipelines
* https://python.langchain.com/docs/modules/memory/types/buffer


### **1. ConversationBufferMemory:**
* This memory allows for storing messages and then extracts the messages in a variable.

* Buffer for storing conversation memory.

* Create a new model by parsing and validating input data from keyword arguments.

* Raises ValidationError if the input data cannot be parsed to form a valid model.

In [39]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import pipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

# Additional configuration parameters
do_sample = True
pad_token_id = tokenizer.eos_token_id
top_k = 50  # You can adjust this value based on your preference
top_p = 0.95  # You can adjust this value based on your preference
temperature = 0.8  # You can adjust this value based on your preference

pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512, do_sample=do_sample, pad_token_id=pad_token_id, top_k=top_k, top_p=top_p, temperature=temperature)
llm = HuggingFacePipeline(pipeline=pipeline)


#Creating a Mental Health Support Prompt Using LangChain
template = """GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: {context}<|end_of_turn|>
GPT4 Correct Assistant: """
prompt = PromptTemplate(template=template, input_variables=["context"])

memory = ConversationBufferMemory(human_prefix="User", ai_prefix="Mental Health Counselor")

conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,  #(setting verbose=True so we can see the prompt
    memory=memory
)

### **Example: 01**

In [40]:
context = "I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it. How can I change my feeling of being worthless to everyone?"
conversation.predict(context=input)



> Entering new LLMChain chain...
Prompt after formatting:
GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: I had to put a restraining order against my ex-fiancé. He was served last night. He was mentally and verbally abusive towards me. I thought it was my nerves, but every time I stand up, I get sick. If I sit and rock, I'm fine.<|end_of_turn|>
GPT4 Correct Assistant: 

> Finished chain.


"\n\nDear [Name],\n\nI'm sorry to hear about the challenging situation you're going through. It's never easy to deal with mental health issues, especially when they're compounded by external stressors like relationships with abusive partners. I'm here to listen and support you in any way I can.\n\nFirst, let's address your physical symptoms. It sounds like you may be experiencing symptoms of anxiety or a panic response when you stand up. These physical symptoms can be quite distressing and can make you feel more anxious, which can create a cycle that's difficult to break. I suggest reaching out to your primary care physician or a mental health professional who can help assess your physical symptoms and provide appropriate care.\n\nAs for the emotional and psychological aspects of your situation, it's essential to recognize that you've taken a significant step by putting a restraining order against your ex-fiancé. This action shows that you're prioritizing your well-being and setting bo

### **Example: 02**

In [41]:
input = "I have a friend that who I used to be in a relationship with. It was brief and turned into us being just good friends. I spent the weekend with him and  it upset my boyfriend. Was i wrong?"
conversation.predict(context=input)



> Entering new LLMChain chain...
Prompt after formatting:
GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: I have a friend that who I used to be in a relationship with. It was brief and turned into us being just good friends. I spent the weekend with him and  it upset my boyfriend. Was i wrong?<|end_of_turn|>
GPT4 Correct Assistant: 

> Finished chain.


"\n\nAs a mental health counselor, I understand that relationships and friendships can be complex and emotionally challenging. It's important to acknowledge and validate your feelings and experiences. Here is some guidance that might be helpful for you:\n\n1. Communication is key: It's essential to communicate openly with your boyfriend about your feelings and the situation. Share with him that you value your relationship and that spending time with your friend was not an intentional attempt to harm your relationship. Encourage open dialogue and listen to his concerns.\n2. Boundaries: It's important to establish healthy boundaries in your relationships. Discuss with your friend the appropriate level of contact and time spent together that both of you are comfortable with. This can help prevent misunderstandings and potential conflicts.\n3. Reflect on your intentions: Think about your intentions when spending time with your friend. If you genuinely had no intention of rekindling a roman

### **Example: 03**

In [42]:
input = "I had to put a restraining order against my ex-fiancé. He was served last night. He was mentally and verbally abusive towards me. I thought it was my nerves, but every time I stand up, I get sick. If I sit and rock, I'm fine."
conversation.predict(context=input)



> Entering new LLMChain chain...
Prompt after formatting:
GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: I had to put a restraining order against my ex-fiancé. He was served last night. He was mentally and verbally abusive towards me. I thought it was my nerves, but every time I stand up, I get sick. If I sit and rock, I'm fine.<|end_of_turn|>
GPT4 Correct Assistant: 

> Finished chain.


"🤗 I'm so glad you reached out for support. It's important to acknowledge the strength it takes to seek help and protect yourself from harmful situations. It's clear that you're going through a tough time, and I'm here to help you navigate this challenging situation.\n\nFirst, I want to validate your feelings and experience. It's completely understandable that you've had to put a restraining order against your ex-fiancé. Mental and verbal abuse can be incredibly damaging and distressing. It's great that you've taken steps to protect yourself and your well-being.\n\nRegarding the physical symptoms you're experiencing when you stand up, it's essential to consult with a healthcare professional to rule out any medical issues. However, it's possible that your body is reacting to stress and anxiety related to your situation. It's common for our bodies to manifest stress in various ways, and it's important to address these feelings to promote your overall well-being.\n\nIn the meantime, here 

In [43]:
#We can also get the history as a list of messages (this is useful if you are using this with a chat model).
memory.load_memory_variables({})

{'history': "User: I had to put a restraining order against my ex-fiancé. He was served last night. He was mentally and verbally abusive towards me. I thought it was my nerves, but every time I stand up, I get sick. If I sit and rock, I'm fine.\nMental Health Counselor: \n\nDear [Name],\n\nI'm sorry to hear about the challenging situation you're going through. It's never easy to deal with mental health issues, especially when they're compounded by external stressors like relationships with abusive partners. I'm here to listen and support you in any way I can.\n\nFirst, let's address your physical symptoms. It sounds like you may be experiencing symptoms of anxiety or a panic response when you stand up. These physical symptoms can be quite distressing and can make you feel more anxious, which can create a cycle that's difficult to break. I suggest reaching out to your primary care physician or a mental health professional who can help assess your physical symptoms and provide appropriat

In [44]:
print(memory.buffer)

User: I had to put a restraining order against my ex-fiancé. He was served last night. He was mentally and verbally abusive towards me. I thought it was my nerves, but every time I stand up, I get sick. If I sit and rock, I'm fine.
Mental Health Counselor: 

Dear [Name],

I'm sorry to hear about the challenging situation you're going through. It's never easy to deal with mental health issues, especially when they're compounded by external stressors like relationships with abusive partners. I'm here to listen and support you in any way I can.

First, let's address your physical symptoms. It sounds like you may be experiencing symptoms of anxiety or a panic response when you stand up. These physical symptoms can be quite distressing and can make you feel more anxious, which can create a cycle that's difficult to break. I suggest reaching out to your primary care physician or a mental health professional who can help assess your physical symptoms and provide appropriate care.

As for the 